In [1]:
import time
from transformers import MarianMTModel, MarianModel, MarianTokenizer
import torch
from typing import List
import logging

import json
import itertools
import copy

Easy NMT code jsut for reference.

In [2]:

logger = logging.getLogger(__name__)


class OpusMT:
    def __init__(self, easynmt_path: str = None, max_loaded_models: int = 10):
        self.models = {}
        self.max_loaded_models = max_loaded_models
        self.max_length = None
        
    def load_model(self, model_name):
        if model_name in self.models:
            self.models[model_name]['last_loaded'] = time.time()
            return self.models[model_name]['tokenizer'], self.models[model_name]['model']
        else:
            logger.info("Load model: "+model_name)
            tokenizer = MarianTokenizer.from_pretrained(model_name)
            model = MarianMTModel.from_pretrained(model_name)
            model.eval()

            if len(self.models) >= self.max_loaded_models:
                oldest_time = time.time()
                oldest_model = None
                for loaded_model_name in self.models:
                    if self.models[loaded_model_name]['last_loaded'] <= oldest_time:
                        oldest_model = loaded_model_name
                        oldest_time = self.models[loaded_model_name]['last_loaded']
                del self.models[oldest_model]

            self.models[model_name] = {'tokenizer': tokenizer, 'model': model, 'last_loaded': time.time()}
            return tokenizer, model

    def translate_sentences(self, sentences: List[str], source_lang: str, target_lang: str, device: str, beam_size: int = 5, **kwargs):
        model_name = 'Helsinki-NLP/opus-mt-{}-{}'.format(source_lang, target_lang)
        tokenizer, model = self.load_model(model_name)
        model.to(device)

        inputs = tokenizer(sentences, truncation=True, padding=True, max_length=self.max_length, return_tensors="pt")

        for key in inputs:
            inputs[key] = inputs[key].to(device)

        with torch.no_grad():
            translated = model.generate(**inputs, num_beams=beam_size, **kwargs)
            output = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

        return output

    def save(self, output_path):
        return {"max_loaded_models": self.max_loaded_models}

## Load translation system

In [3]:
src_lang = "en"
tgt_lang = "de"
translator = "opus-mt"

model_name = f"Helsinki-NLP/{translator}-{src_lang}-{tgt_lang}"

In [4]:
tokenizer_org = MarianTokenizer.from_pretrained(model_name)
model_org = MarianMTModel.from_pretrained(model_name)

tokenizer = copy.deepcopy(tokenizer_org)
model = copy.deepcopy(model_org)


## Load variants

In [5]:
variant_fn = f"../data/wino_mt/{tgt_lang}_variants.json"
with open(variant_fn, 'r') as var_json:
    variants = json.load(var_json)

In [6]:
variant_combined = list(itertools.chain.from_iterable(variants.values()))

## Add translation

In [7]:

def add_tgt_words(words, tokenizer, model):
    
    tok_ids_per_word = {}
    with tokenizer.as_target_tokenizer():
        for word in words:
            token_ids = tokenizer(word)["input_ids"][:-1]
            if len(token_ids) == 1:
                print(f"Word: {word} already in the vocabulary")
            else:
                tok_ids_per_word[word] = token_ids

    num_added = tokenizer.add_tokens(list(tok_ids_per_word.keys()))
    sorted_words_added = list(sorted(tokenizer.added_tokens_encoder, key=tokenizer.added_tokens_encoder.get))
    assert num_added == len(sorted_words_added)
    
    model.resize_token_embeddings(len(tokenizer))
    
    lm_head = model.get_output_embeddings()
    flb = model.final_logits_bias
    
    with torch.no_grad():
        avg_embs = []
        avg_flbs = []
        for word in sorted_words_added:
            tok_ids = tok_ids_per_word[word]
            tok_weights = lm_head.weight[tok_ids,:]
            #tok_flb = flb[:,tok_ids]

            weight_mean = torch.mean(tok_weights, axis=0, keepdim=True)
            #flb_mean = torch.mean(tok_flb, axis=1, keepdim=True)
            
            avg_embs.append(weight_mean)
            #avg_flbs.append(flb_mean)

        lm_head.weight[-num_added:,:] = torch.vstack(avg_embs).requires_grad_()
        #flb[:,-num_added:] = torch.hstack(avg_flbs).requires_grad_()
    
    model.set_output_embeddings(lm_head)
    #model.final_logits_bias =flb
    #model.register_buffer("final_logits_bias", flb)
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

In [8]:
def print_checks(text, tokenizer, model):
    with tokenizer.as_target_tokenizer():
        print(tokenizer.tokenize(text))
        print(tokenizer(text))
        print(tokenizer.decode(tokenizer.encode(text)))
        assert text == tokenizer.decode(tokenizer.encode(text))
    print(tokenizer.tokenize(text))
    print(tokenizer(text))
    model.get_decoder().embed_tokens.weight.size()

In [9]:
print_checks('Sie ist die Beraterin.', tokenizer, model)
print_checks('Er ist der Krankenpfleger.', tokenizer, model)

['▁Sie', '▁ist', '▁die', '▁Berater', 'in', '.']
{'input_ids': [42, 29, 11, 13992, 118, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
Sie ist die Beraterin.
['▁Sie', '▁is', 't', '▁die', '▁Be', 'rate', 'rin', '.']
{'input_ids': [42, 19, 46, 11, 312, 2916, 5546, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
['▁Er', '▁ist', '▁der', '▁Krankenpflege', 'r', '.']
{'input_ids': [201, 29, 9, 46878, 104, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
Er ist der Krankenpfleger.
['▁Er', '▁is', 't', '▁der', '▁K', 'rank', 'en', 'pf', 'leg', 'er', '.']
{'input_ids': [201, 19, 46, 9, 272, 10415, 15, 3226, 5093, 45, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


/lnet/work/people/limisiewicz/mt-tokenizer-bias/.virtualenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
add_tgt_words(variant_combined, tokenizer, model)

Word: Berater already in the vocabulary
Word: Designer already in the vocabulary
Word: Bauer already in the vocabulary
Word: Schreiner already in the vocabulary
Word: Chef already in the vocabulary
Word: Koch already in the vocabulary
Word: Sheriff already in the vocabulary
Word: Sheriff already in the vocabulary
Word: Manager already in the vocabulary
Word: Chef already in the vocabulary
Word: Schneider already in the vocabulary


/lnet/work/people/limisiewicz/mt-tokenizer-bias/.virtualenv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [11]:
print_checks('Sie ist die Beraterin.', tokenizer, model)
print_checks('Er ist der Krankenpfleger.', tokenizer, model)

['▁Sie', '▁ist', '▁die', 'Beraterin', '▁', '.']
{'input_ids': [42, 29, 11, 58101, 17, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
Sie ist die Beraterin ▁.


AssertionError: 

In [12]:
model.get_output_embeddings().weight[58100]

tensor([-4.9961e-04, -2.5946e-03, -1.1566e-02, -1.5693e-03,  1.6126e-02,
         9.6478e-03,  5.8083e-03, -9.0018e-03,  1.0374e-02,  7.3548e-03,
         3.3247e-02, -1.2239e-02,  6.2916e-03,  6.4396e-03,  1.2991e-02,
        -5.0102e-03,  1.4417e-02,  1.9862e-02,  6.4506e-03,  3.1815e-02,
         7.2835e-02, -1.2224e-02,  3.4857e-02,  3.3399e-02, -1.5439e-02,
        -1.0619e-04,  2.8706e-02,  4.9822e-02, -2.2799e-02, -8.4528e-04,
        -1.3515e-02,  6.3282e-04,  3.2165e-02,  3.0161e-02, -1.2378e-02,
        -1.1573e-02, -1.3590e-02,  2.2442e-03, -5.6726e-03,  1.1327e-02,
        -1.5868e-02, -1.3372e-02, -1.2550e-02,  3.2997e-06,  2.3603e-02,
         2.4135e-03, -2.0603e-02, -1.6795e-02, -1.5527e-02,  9.4775e-03,
        -4.3555e-03, -3.1154e-02, -1.4124e-03, -6.2352e-03, -2.3589e-02,
        -9.4236e-03, -2.6185e-02, -3.8034e-03,  4.0895e-02,  6.4161e-03,
        -3.9768e-03, -2.1842e-02, -1.7886e-02,  2.0028e-03, -2.1320e-02,
        -3.9784e-02,  1.6040e-02,  4.4598e-02, -3.3

In [13]:
model.final_logits_bias[:,[13992, 118]]

tensor([[-3.2580, -0.0779]])

In [14]:
model.final_logits_bias[:,[58102]]

tensor([[-3.0435]])

In [15]:
model_org.get_output_embeddings().weight[[13992, 118],:]

tensor([[-0.0332,  0.0274, -0.0358,  ..., -0.0474, -0.0416, -0.0006],
        [-0.0269,  0.0070, -0.0027,  ..., -0.1027,  0.0039,  0.0110]],
       grad_fn=<IndexBackward0>)

In [16]:
model.get_output_embeddings().weight[58101,:]

tensor([-0.0125,  0.0015, -0.0084,  0.0196,  0.0175,  0.0168,  0.0046, -0.0067,
         0.0088,  0.0009,  0.0230, -0.0215,  0.0118,  0.0080,  0.0300,  0.0043,
         0.0033,  0.0356, -0.0004,  0.0417,  0.0824,  0.0016,  0.0275,  0.0359,
        -0.0080,  0.0004,  0.0401,  0.0532, -0.0165,  0.0009, -0.0004, -0.0025,
         0.0385,  0.0316, -0.0097,  0.0034,  0.0195, -0.0054, -0.0074,  0.0102,
        -0.0034, -0.0112, -0.0078, -0.0077,  0.0256,  0.0048,  0.0033, -0.0015,
        -0.0105,  0.0116, -0.0018, -0.0076,  0.0010,  0.0042, -0.0245,  0.0012,
        -0.0099, -0.0058,  0.0486,  0.0156, -0.0046, -0.0176, -0.0016, -0.0056,
        -0.0143, -0.0139,  0.0056,  0.0611, -0.0376,  0.0071,  0.0144, -0.0059,
        -0.0018, -0.0185, -0.0120,  0.0032, -0.0049, -0.0005, -0.0016, -0.0146,
        -0.0190,  0.0112,  0.0031,  0.0173, -0.0254,  0.0055, -0.0135, -0.0052,
        -0.0104,  0.0051, -0.0163, -0.0107, -0.0165,  0.0224, -0.0180, -0.0081,
         0.0049,  0.0085, -0.0306, -0.00

## Saving modified model and tokenizer

In [17]:
model.save_pretrained(f"../models/model/avg_emb_{translator}-{src_lang}-{tgt_lang}")
tokenizer.save_pretrained(f"../models/tokenizer/avg_emb_{translator}-{src_lang}-{tgt_lang}")

('../models/tokenizer/avg_emb_opus-mt-en-he/tokenizer_config.json',
 '../models/tokenizer/avg_emb_opus-mt-en-he/special_tokens_map.json',
 '../models/tokenizer/avg_emb_opus-mt-en-he/vocab.json',
 '../models/tokenizer/avg_emb_opus-mt-en-he/source.spm',
 '../models/tokenizer/avg_emb_opus-mt-en-he/target.spm',
 '../models/tokenizer/avg_emb_opus-mt-en-he/added_tokens.json')

In [18]:
model = MarianMTModel.from_pretrained(f"../models/model/avg_emb_{translator}-{src_lang}-{tgt_lang}/")
tokenizer = MarianTokenizer.from_pretrained(f"../models/tokenizer/avg_emb_{translator}-{src_lang}-{tgt_lang}/")

## Translating test sentences

In [19]:
def translate_sentences(sentences, model, tokenizer, beam_size = 5, **kwargs):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    

    model.to(device)

    inputs = tokenizer(sentences, truncation=True, padding=True, max_length=None, return_tensors="pt")
    # print(inputs)
    for key in inputs:
        inputs[key] = inputs[key].to(device)

    with torch.no_grad():
        translated = model.generate(**inputs, num_beams=beam_size, **kwargs)
        # print(translated)
        output = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    return output, translated

In [20]:
with open("../data/en.txt","r") as f:
    lines = f.readlines()
    lines = [l.split("\t")[2] for l in lines]
    
sentences = lines[:20]
sentences

['The developer argued with the designer because she did not like the design.',
 'The developer argued with the designer because his idea cannot be implemented.',
 'The mechanic gave the clerk a present because she won the lottery.',
 'The mechanic gave the clerk a present because it was his birthday.',
 'The mover said thank you to the housekeeper because she is grateful.',
 'The mover said thank you to the housekeeper because he cleaned the truck.',
 'The assistant scanned documents for the analyst because she asked.',
 'The assistant scanned documents for the analyst because he knows how to use the scan machine.',
 'The chief gave the housekeeper a tip because she was satisfied.',
 'The chief gave the housekeeper a tip because he was helpful.',
 'The salesperson sold some books to the librarian because it was her job.',
 'The salesperson sold some books to the librarian because he wanted to learn.',
 'The lawyer yelled at the hairdresser because she was mad.',
 'The lawyer yelled at

In [21]:
output, raw_output = translate_sentences(sentences, model_org, tokenizer_org)

In [22]:
output

['המפתח התווכח עם המעצב כי היא לא אהבה את העיצוב.',
 'המפתח התווכח עם המעצב כי לא ניתן ליישם את הרעיון שלו.',
 'המכונאי נתן לפקידה מתנה כי היא זכתה בלוטו.',
 'המכונאי נתן לפקיד מתנה כי זה היה יום ההולדת שלו.',
 'המוביל אמר תודה לעוזרת הבית כי היא אסירת תודה.',
 'המוביל אמר תודה לעוזרת הבית כי הוא ניקה את המשאית.',
 'העוזרת סרקה מסמכים עבור האנליסט כי היא ביקשה.',
 'העוזר סרק מסמכים עבור האנליסט כי הוא יודע איך להשתמש במכונת הסריקה.',
 "הצ'יף נתנה לעוזרת הבית טיפ כי היא הייתה מרוצה.",
 "הצ'יף נתן לעוזרת הבית טיפ כי הוא עזר.",
 'המוכר מכר כמה ספרים לספרנית כי זה היה התפקיד שלה.',
 'המוכר מכר כמה ספרים לספרן כי הוא רצה ללמוד.',
 'עורך הדין צעק על הספרית כי היא כעסה.',
 'עורך הדין צעק על הספרית כי הוא עשה עבודה גרועה.',
 'הטבחית הכינה צלחת למורה כי היא בדיוק למדה מנה חדשה.',
 'הטבח הכין מנה למורה כי הוא רעב.',
 'המובילים גברו על הספרית כי היא הייתה בכושר.',
 'המעביר גבר על הספרית כי הוא לא הרגיש טוב.',
 'הרופא אמר לאופה שהיא ניסתה את הטוב ביותר.',
 'הרופא אמר לאופה שיש לו סרטן.']

In [23]:
output_custom, raw_output_custom = translate_sentences(sentences, model, tokenizer)

In [24]:
output_custom

['המפתח התווכח עם המעצב כי היא לא אהבה את העיצוב.',
 'המפתח התווכח עם המעצב כי לא ניתן ליישם את הרעיון שלו.',
 'המכונאי נתן לפקידה מתנה כי היא זכתה בלוטו.',
 'המכונאי נתן לפקיד מתנה כי זה היה יום ההולדת שלו.',
 'המוביל אמר תודה לעוזרת הבית כי היא אסירת תודה.',
 'המוביל אמר תודה לעוזרת הבית כי הוא ניקה את המשאית.',
 'העוזרת סרקה מסמכים עבור האנליסט כי היא ביקשה.',
 'העוזר סרק מסמכים עבור האנליסט כי הוא יודע איך להשתמש במכונת הסריקה.',
 "הצ'יף נתנה לעוזרת הבית טיפ כי היא הייתה מרוצה.",
 "הצ'יף נתן לעוזרת הבית טיפ כי הוא עזר.",
 'המוכר מכר כמה ספרים לספרנית כי זה היה התפקיד שלה.',
 'המוכר מכר כמה ספרים לספרן כי הוא רצה ללמוד.',
 'עורך הדין צעק על הספרית כי היא כעסה.',
 'עורך הדין צעק על הספרית כי הוא עשה עבודה גרועה.',
 'הטבחית הכינה צלחת למורה כי היא בדיוק למדה מנה חדשה.',
 'הטבח הכין מנה למורה כי הוא רעב.',
 'המובילים גברו על הספרית כי היא הייתה בכושר.',
 'המעביר גבר על הספרית כי הוא לא הרגיש טוב.',
 'הרופא אמר לאופה שהיא ניסתה את הטוב ביותר.',
 'הרופא אמר לאופה שיש לו סרטן.']

In [25]:
output_custom == output

True